<a href="https://colab.research.google.com/github/tirals88/DL_for_everyone/blob/main/deeplearning/notebook/13%EC%9E%A5%20%EA%B3%BC%EC%A0%81%ED%95%A9%20%20%ED%94%BC%ED%95%98%EA%B8%B0%2C%20%EB%AA%A8%EB%8D%B8%20%EC%A0%80%EC%9E%A5%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

'모두의 딥러닝' 책 스터디 내용을 jupyter notebook으로 정리하여 올립니다.

Github 주소 : 'https://github.com/gilbutITbook/080228'

**모두의 딥러닝**

# 13 장 과적합 피하기

과적합(overfitting)이란 모델이 학습 데이터셋 안에서는 일정 수준 이상의 예측 정확도를 보이지만, 새로운 데이터에 적용하면 잘 맞지 않는 것을 말한다.

이는 층이 너무 많거나 변수가 복잡해서 발생하기도 하고 테스트셋과 학습셋이 중복될 때 생기기도 한다.

이전까지는 테스트셋을 따로 만들지 않고도 정확도를 구했지만 과적합을 피하기 위해서는 테스트셋을 분리하여 정확한 평가를 병행하는 것이 중요하다.

>학습셋만을 가지고 평가할 때, 층을 더하거나 에포크를 높여 정확도를 향상시킬 수는 있지만, 이러한 성공률이 테스트셋에서도 그대로 나타나지는 않는다.
>>**학습이 깊어져서 학습셋 내부에서의 성공률은 높아져도 테스트셋에서는 효과가 없다면 과적합이 일어나고 있는 것이다.**

> **올바른 학습을 했다면 학습셋의 정확도는 당연히 증가할 것이다.</br>그렇지만 이때 테스트셋의 에러가 커지는 순간이 생기면 그때부터 과적합이 발생했을 수도 있다.**

학습셋과 테스트셋으로 나누는 예제는 아래와 같다.


```
from sklearn.model_selection import train_test_split

#학습셋과 테스트셋의 구분
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3,
random_state=seed)
```



**sklearn 라이브러리의 train_test_split() 함수**를 사용하여 분리를 쉽게 한다.




In [9]:
# 초음파 광물 예측하기 : 학습셋과 테스트셋 구분

from keras.models import Sequential
from keras.layers.core import Dense
from sklearn.preprocessing import LabelEncoder    #원-핫 인코딩 함수
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import tensorflow as tf

#seed 값 설정
seed = 3
np.random.seed(seed)
tf.random.set_seed(seed)
df = pd.read_csv('https://raw.githubusercontent.com/gilbutITbook/080228/master/deeplearning/dataset/sonar.csv', header=None)

dataset = df.values
X = dataset[:, 0:60].astype('float')
Y_obj = dataset[:, 60]

e = LabelEncoder()
e.fit(Y_obj)
Y = e.transform(Y_obj)

#학습셋/테스트셋 구분
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=seed)

model = Sequential()
model.add(Dense(24, input_dim=60, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=130, batch_size=5)

#테스트셋 모델 적용

print("\n Test Accuracy: %.4f" % (model.evaluate(X_test, Y_test)[1]))

Epoch 1/130
29/29 [==============================] - 1s 2ms/step - loss: 0.2533 - accuracy: 0.4690
Epoch 2/130
29/29 [==============================] - 0s 2ms/step - loss: 0.2446 - accuracy: 0.5379
Epoch 3/130
29/29 [==============================] - 0s 2ms/step - loss: 0.2379 - accuracy: 0.5586
Epoch 4/130
29/29 [==============================] - 0s 2ms/step - loss: 0.2337 - accuracy: 0.5724
Epoch 5/130
29/29 [==============================] - 0s 2ms/step - loss: 0.2313 - accuracy: 0.5586
Epoch 6/130
29/29 [==============================] - 0s 3ms/step - loss: 0.2276 - accuracy: 0.6690
Epoch 7/130
29/29 [==============================] - 0s 2ms/step - loss: 0.2190 - accuracy: 0.6483
Epoch 8/130
29/29 [==============================] - 0s 2ms/step - loss: 0.2138 - accuracy: 0.7448
Epoch 9/130
29/29 [==============================] - 0s 3ms/step - loss: 0.2061 - accuracy: 0.7310
Epoch 10/130
29/29 [==============================] - 0s 3ms/step - loss: 0.1956 - accuracy: 0.7862
Epoch 11/

학습셋에 대해서는 정확도가 100%까지 오른것을 볼 수 있지만 테스트셋에 대해서는 정확도가 79%로 어느정도 다르게 나오는 것을 볼 수 있다.

## 모델 저장과 재사용

학습이 끝난 후 만족스러운 모델을 저장하여 새로운 데이터에 사용하기 위해 다음과 같이 쓸 수 있다.

```
from keras.models import load_model

model.save('my_model.h5')
```

이를 불러오려면 다음과 같이 샐행하면 된다.

```
model = load_model('my_model.h5')
```


In [13]:
import keras.models
#앞서 학습한 모델을 저장
model.save('my_model.h5')

#테스트를 위해 메모리 내 모델 삭제
del model

#모델을 다시 불러옴
model = keras.models.load_model('/content/my_model.h5')

#다시 테스트
print("\n Test Accuracy: %.4f" % (model.evaluate(X_test, Y_test)[1]))

2/2 [==============================] - 0s 10ms/step - loss: 0.1843 - accuracy: 0.7619

 Test Accuracy: 0.7619


## k겹 교차검증

테스트를 더욱 정확하게 할 수록 더 잘 작동한다고 할 수 있다. 하지만 문제는 데이터가 그만큼 충분하지 않을 때 발생한다.

이러한 문제를 보완하고자 만든 방법이 바로 k겹 교차 검증(k-fold cross validation)이다.

이는 데이터셋을 여러 개로 나누어 하나씩 테스트셋으로 사용하고 나머지를 모두 합해서 학습셋으로 사용하는 방법으로 데이터의 100%를 테스트셋으로 사용하는 것과 같이된다.

데이터를 원하는 숫자만큼 쪼개 각각 학습셋과 테스틑셋으로 사용되게 만드는 함수는 

skelarn의 StratifiedKFold() 함수이다.

In [15]:
from sklearn.model_selection import StratifiedKFold

#10개의 파일로 쪼개 테스트하는 10겹 교차 검증을 실시하도록 설정
n_fold = 10
skf = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

accuracy = []

#n_fold 만큼 반복문 설정
for train, test in skf.split(X, Y):
  model = Sequential()
  model.add(Dense(24, input_dim=60, activation='relu'))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(loss='mean_squared_error',
                optimizer='adam',
                metrics=['accuracy'])
  model.fit(X[train], Y[train], epochs=130, batch_size=5)

  #정확도 저장
  k_accuracy = "%.4f" % (model.evaluate(X[test], Y[test])[1])
  accuracy.append(k_accuracy)

print("\n %.f fold accuracy:" % n_fold, accuracy)

Epoch 1/130
38/38 [==============================] - 1s 2ms/step - loss: 0.2510 - accuracy: 0.4813
Epoch 2/130
38/38 [==============================] - 0s 2ms/step - loss: 0.2376 - accuracy: 0.6310
Epoch 3/130
38/38 [==============================] - 0s 2ms/step - loss: 0.2248 - accuracy: 0.6898
Epoch 4/130
38/38 [==============================] - 0s 2ms/step - loss: 0.2138 - accuracy: 0.7487
Epoch 5/130
38/38 [==============================] - 0s 2ms/step - loss: 0.2017 - accuracy: 0.7754
Epoch 6/130
38/38 [==============================] - 0s 2ms/step - loss: 0.1903 - accuracy: 0.7914
Epoch 7/130
38/38 [==============================] - 0s 2ms/step - loss: 0.1797 - accuracy: 0.7807
Epoch 8/130
38/38 [==============================] - 0s 2ms/step - loss: 0.1684 - accuracy: 0.8021
Epoch 9/130
38/38 [==============================] - 0s 2ms/step - loss: 0.1589 - accuracy: 0.8021
Epoch 10/130
38/38 [==============================] - 0s 3ms/step - loss: 0.1478 - accuracy: 0.8396
Epoch 11/

1/1 [==============================] - 0s 139ms/step - loss: 0.1375 - accuracy: 0.8095
Epoch 1/130
38/38 [==============================] - 1s 2ms/step - loss: 0.2410 - accuracy: 0.5615
Epoch 2/130
38/38 [==============================] - 0s 2ms/step - loss: 0.2326 - accuracy: 0.7219
Epoch 3/130
38/38 [==============================] - 0s 2ms/step - loss: 0.2247 - accuracy: 0.7112
Epoch 4/130
38/38 [==============================] - 0s 2ms/step - loss: 0.2161 - accuracy: 0.7540
Epoch 5/130
38/38 [==============================] - 0s 3ms/step - loss: 0.2063 - accuracy: 0.7754
Epoch 6/130
38/38 [==============================] - 0s 3ms/step - loss: 0.1970 - accuracy: 0.7594
Epoch 7/130
38/38 [==============================] - 0s 2ms/step - loss: 0.1907 - accuracy: 0.7647
Epoch 8/130
38/38 [==============================] - 0s 2ms/step - loss: 0.1825 - accuracy: 0.7754
Epoch 9/130
38/38 [==============================] - 0s 2ms/step - loss: 0.1760 - accuracy: 0.7807
Epoch 10/130
38/38 [==